In [32]:
import numpy as np
import scipy.io
import h5py
import sklearn.neural_network

In [31]:
#hdf5_data = h5py.File('MRS_forDL_120617.mat')
hdf5_data = h5py.File('MRS_CSI_DL.mat')
print 'Keys:', hdf5_data.keys()

Keys: [u'X_test', u'X_tr_ns', u'Y_cr_tr', u'Y_cr_ts', u'Y_naa_tr', u'Y_naa_ts', u'Y_pch_tr', u'Y_pch_ts']


In [42]:
X_train = np.array(hdf5_data['X_tr_ns']).transpose((1,0))
print 'X_train:', X_train.shape

X_test = np.array(hdf5_data['X_test']).transpose((1,0))
print 'X_test:', X_test.shape 

Y_naa_tr = np.array(hdf5_data['Y_naa_tr']).transpose((1,0))
print 'Y_naa_tr:', Y_naa_tr.shape

Y_naa_ts = np.array(hdf5_data['Y_naa_ts']).transpose((1,0))
print 'Y_naa_ts:', Y_naa_ts.shape

Y_pch_tr = np.array(hdf5_data['Y_pch_tr']).transpose((1,0))
print 'Y_pch_tr:', Y_pch_tr.shape

Y_pch_ts = np.array(hdf5_data['Y_pch_ts']).transpose((1,0))
print 'Y_pch_ts:', Y_pch_ts.shape

#Y_cr_tr = np.array(hdf5_data['Y_cr_tr']).transpose((1,0))
#print 'Y_cr_tr:', Y_cr_tr.shape

#Y_cr_ts = np.array(hdf5_data['Y_cr_ts']).transpose((1,0))
#print 'Y_cr_ts:', Y_cr_ts.shape

X_train: (1044014, 263)
X_test: (10546, 263)
Y_naa_tr: (1044014, 1)
Y_naa_ts: (10546, 1)
Y_pch_tr: (1044014, 1)
Y_pch_ts: (10546, 1)
Y_cr_tr: (1044014, 1)
Y_cr_ts: (10546, 1)


In [43]:
# Train data
Y_train = np.concatenate((Y_naa_tr, Y_pch_tr, Y_cr_tr),axis = 1)
print 'Y_train:', Y_train.shape 
# Test data
Y_test = np.concatenate((Y_naa_ts, Y_pch_ts, Y_cr_ts),axis = 1)
print 'Y_test:', Y_test.shape

Y_train: (1044014, 3)
Y_test: (10546, 3)


In [47]:
# Multilayer perceptron
regressor = sklearn.neural_network.MLPRegressor(hidden_layer_sizes=(100,100,100,), 
                                                activation='relu', 
                                                solver='adam', 
                                                learning_rate_init = 0.001, 
                                                max_iter=10000,
                                                momentum=0.9, 
                                                nesterovs_momentum=True, 
                                                early_stopping=True, 
                                                validation_fraction=0.1,
                                                verbose=True)

In [48]:
regressor.fit(X_train, Y_train)

Iteration 1, loss = 0.10713412
Validation score: 0.958787
Iteration 2, loss = 0.05083925
Validation score: 0.964803
Iteration 3, loss = 0.04709373
Validation score: 0.967245
Iteration 4, loss = 0.04492102
Validation score: 0.970317
Iteration 5, loss = 0.04399726
Validation score: 0.962837
Iteration 6, loss = 0.04284135
Validation score: 0.962651
Iteration 7, loss = 0.04255467
Validation score: 0.963671
Validation score did not improve more than tol=0.000100 for two consecutive epochs. Stopping.


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(100, 100, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=10000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=True, warm_start=False)

In [49]:
Y_predicted = regressor.predict(X_test)

In [52]:
# Error in prediction
error = np.mean((Y_test - Y_predicted)**2)
print error

0.1063033824


In [53]:
# Randomly sample 10 elements
idxs = np.random.choice(X_test.shape[0], 10, replace=False)
for idx in idxs:
    print 'Index', idx
    print 'Actual:', Y_test[idx,:]
    print 'Predicted', Y_predicted[idx,:]

Index 9732
Actual: [ 1.83333337  1.16666663  1.        ]
Predicted [ 1.7599638   1.11776961  1.00174551]
Index 959
Actual: [ 0.33333334  1.33333337  1.        ]
Predicted [ 0.38723439  1.23683883  1.01124037]
Index 4468
Actual: [ 2.   1.5  1. ]
Predicted [ 2.22777743  1.53253037  0.98571432]
Index 8791
Actual: [ 1.5   0.25  1.  ]
Predicted [ 1.44170874  0.21014109  0.9922208 ]
Index 1106
Actual: [ 0.5   0.75  1.  ]
Predicted [ 0.44933315  0.70811092  0.97656984]
Index 9806
Actual: [ 0.54545456  0.90909094  1.        ]
Predicted [ 0.52401347  0.86405801  0.98917228]
Index 3697
Actual: [ 0.27272728  0.54545456  1.        ]
Predicted [ 0.23405641  0.53354579  0.9855922 ]
Index 6884
Actual: [ 0.5         1.10000002  1.        ]
Predicted [ 0.5315074   1.10245318  0.99219991]
Index 7622
Actual: [ 0.90909094  0.90909094  1.        ]
Predicted [ 0.92260346  0.88691785  1.0319793 ]
Index 4827
Actual: [ 0.          3.33333325  1.        ]
Predicted [ 0.16296436  3.72601281  1.0156793 ]


In [54]:
# Write as a mat file
predictions_dict = {'Y_naa_pred': Y_predicted[:,0],
               'Y_pch_pred': Y_predicted[:,1],
               'Y_cr_pred': Y_predicted[:,2]}

scipy.io.savemat('predictions.mat', predictions_dict)